In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


path = "C:\\Proganiy\\pp-laba3\\annotation.csv"

In [2]:
def create_ann(annatation: str) -> pandas.DataFrame:

    """Создаёт датафрейм по пути аннатации"""

    frame = pandas.DataFrame(columns =["Оценка","Kоличество слов","Текст рецензии"])
    ann_temp = open(annatation, "r", encoding="utf-8")
    for otzv in ann_temp.readlines():
        mas_otzv = otzv.split(",")
        otzv_temp = open(mas_otzv[0],"r",encoding="utf-8")
        otzv_text = " ".join(otzv_temp)
        row = pandas.Series({"Оценка": int(mas_otzv[2]),"Kоличество слов": len(otzv_text), "Текст рецензии": otzv_text})
        new_row = pandas.DataFrame([row], columns=frame.columns)
        frame = pandas.concat([frame, new_row], ignore_index=True)
    frame.dropna()
    return frame

data = create_ann(path)
print(data)

     Оценка Kоличество слов                                     Текст рецензии
0         1            1853  11 марта этого года переводила 145 в Германию(...
1         1            4278  В пятницу, 22 апреля 2022 г. Сбербанк заблокир...
2         1            2035  Здравствуйте. Перед оформлением досудебной пре...
3         1            3503  Я являюсь клиентом Сбербанка уже более десяти ...
4         1            3340  Призываю предпринимателей подумать дважды, пре...
...     ...             ...                                                ...
2995      5             304  Сегодня хочу рассказать о Сбербанке. В этом пр...
2996      5             863  Банк Сбербанк - это прекрасное финансовое учре...
2997      5             979  Пользуюсь сбербанком сколько себя помню, зарпл...
2998      5            2084  Я не сразу научилась пользоваться кредитной сб...
2999      5             777  Пользуюсь сбербанком давно, никаких нарекании ...

[3000 rows x 3 columns]


In [4]:
def clean (frame: pandas.DataFrame) -> pandas.DataFrame:
    remove_non_alphabets =lambda x: re.sub(r'[^а-яА-Я]',' ',x)

    frame["Текст рецензии"] = frame["Текст рецензии"].apply(remove_non_alphabets)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(Mystem().lemmatize)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(lambda x: ' '.join(x))
    return frame

data = clean(data)
print(data)

     Оценка Kоличество слов                                     Текст рецензии
0         1            1853      март   этот   год   переводить       в   г...
1         1            4278  в   пятница       апрель        г    сбербанк ...
2         1            2035  здравствовать    перед   оформление   досудебн...
3         1            3503  я   являться   клиент   сбербанк   уже   много...
4         1            3340  призывать   предприниматель   подумать   дважд...
...     ...             ...                                                ...
2995      5             304  сегодня   хотеть   рассказывать   о   сбербанк...
2996      5             863  банк   сбербанк     это   прекрасный   финансо...
2997      5             979  пользоваться   сбербанк   сколько   себя   пом...
2998      5            2084  я   не   сразу   научаться   пользоваться   кр...
2999      5             777  пользоваться   сбербанк   давно    никакой   н...

[3000 rows x 3 columns]


In [20]:
max_words = 10000
cv = CountVectorizer(max_features=max_words, stop_words='russian')
sparse_matrix = cv.fit_transform(data["Текст рецензии"]).toarray()

sparse_matrix.shape

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got 'russian' instead.

In [14]:
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(frame["Оценка"]))

class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 1000)
        self.linear2 = nn.Linear(1000, 100)
        self.linear3 = nn.Linear(100, 5)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

model = LogisticRegression()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)


x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
x_test = Variable(torch.from_numpy(x_test)).float()
y_test = Variable(torch.from_numpy(y_test)).long()

NameError: name 'train_test_split' is not defined

In [16]:


epochs = 20
model.train()
loss_values = []
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss_values.append(loss.item())
    pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
    acc = pred * 100.0 / len(x_train)
    print('Epoch: {}, Loss: {}, Accuracy: {}%'.format(epoch+1, loss.item(), acc.numpy()))
    loss.backward()
    optimizer.step()

NameError: name 'model' is not defined

In [ ]:


model.eval()
with torch.no_grad():
    y_pred = model(x_test)
    loss = criterion(y_pred, y_test)
    pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
    print ("Accuracy : {}%".format(100*pred/len(x_test)))